In [1]:
import pandas as pd # импортируем необходимые библиотеки
import numpy as np
import re
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier, GradientBoostingRegressor, ExtraTreesRegressor, BaggingRegressor

from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import KFold
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.tree import ExtraTreeRegressor


from sklearn.cluster import KMeans
from sklearn.feature_extraction.text import TfidfVectorizer
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.decomposition import PCA
from sklearn.decomposition import TruncatedSVD
from tqdm import tqdm
%matplotlib inline

In [2]:
def mape(y_true, y_pred): # необходимая метрика оценки качества
    return np.mean(np.abs((y_pred-y_true)/y_true))

In [3]:
RANDOM_SEED = 13  # фиксируем рандомные параметры и размер валидационной выборки выборки
VAL_SIZE   = 0.33
n_folds = 5
iterations = 1300
learning_rate = 0.05
VERSION = 880207

In [4]:
asd_99 = pd.read_csv('/kaggle/input/car-class-06-03-13-04/new_data_99_06_03_13_04.csv', index_col = 0) # открываем датасет и сразу удаляем дубликаты 
print(asd_99.shape)
asd_99 = asd_99.drop_duplicates().reset_index(drop = True)
print('Размер трейна: ', asd_99.shape, '\n',
      'Названия столбцов трейна: ', ', '.join(asd_99.columns))
a = pd.read_csv('/kaggle/input/sf-dst-car-price/test.csv')
print('Размер теста: ', a.shape, '\n',
      'Названия столбцов теста: ', ', '.join(a.columns))
sample_submission = pd.read_csv('/kaggle/input/sf-dst-car-price/sample_submission.csv')

(109080, 21)
Размер трейна:  (109080, 21) 
 Названия столбцов трейна:  bodyType, brand, color, fuelType, modelDate, name, numberOfDoors, productionDate, vehicleConfiguration, engineDisplacement, enginePower, description, mileage, Комплектация, Привод, Руль, Владельцы, ПТС, Таможня, Владение, Price
Размер теста:  (35990, 23) 
 Названия столбцов теста:  bodyType, brand, color, fuelType, modelDate, name, numberOfDoors, productionDate, vehicleConfiguration, vehicleTransmission, engineDisplacement, enginePower, description, mileage, Комплектация, Привод, Руль, Состояние, Владельцы, ПТС, Таможня, Владение, id


In [5]:
def param_data(data): # посмотрим на данные
  param = pd.DataFrame({
              'dtypes': data.dtypes.values,
              'nunique': data.nunique().values,
              'isna': data.isna().sum().values,
              'loc[0]': data.loc[0].values,
              }, 
             index = data.loc[0].index)
  return param

In [6]:
pd.concat([param_data(asd_99), param_data(a)], axis=1, sort=False)

,dtypes,nunique,isna,loc[0],dtypes,nunique,isna,loc[0]
bodyType,object,112.0,1.0,Седан,object,23.0,0.0,внедорожник 5 дв.
brand,object,13.0,0.0,AUDI,object,13.0,0.0,MERCEDES
color,object,16.0,0.0,040001,object,16.0,0.0,синий
fuelType,object,6.0,0.0,бензин,object,5.0,0.0,бензин
modelDate,float64,70.0,1.0,1990,float64,51.0,0.0,2015
name,object,3490.0,1.0,2.3 MT (133 л.с.),object,2528.0,0.0,250 2.0 AT (211 л.с.) 4WD
numberOfDoors,float64,5.0,1.0,4,float64,4.0,0.0,5
productionDate,int64,73.0,0.0,1991,float64,49.0,0.0,2017
vehicleConfiguration,object,4.0,1.0,MECHANICAL,object,597.0,0.0,ALLROAD_5_DOORS AUTOMATIC 2.0
engineDisplacement,object,431.0,1.0,2.3,object,53.0,0.0,2.0 LTR


In [7]:
def train_(train):
    train = train.drop(['Таможня'], axis = 1)
  # bodyType, оставим только названия кузова как в тесте
    train['bodyType'] = train['bodyType'].apply(lambda x: str(x))
    def bodyType(row):
        for body_type in ['внедорожник 5 дв.', 'седан-хардтоп', 'хэтчбек 5 дв.','внедорожник 3 дв.', 
                      'купе-хардтоп', 'внедорожник открытый','хэтчбек 3 дв.', 'пикап двойная кабина',
                      'пикап полуторная кабина', 'пикап одинарная кабина', 'седан 2 дв.','универсал 5 дв.', 
                      'родстер', 'кабриолет','фургон', 'микровэн','минивэн', 'компактвэн','лифтбек',
                      'купе','тарга', 'седан','лимузин']:
            if row.lower().startswith(body_type):
                return body_type
    train['bodyType'] = train['bodyType'].apply(bodyType)
  # color, сделаем знчения как в тесте
    train['color'] = train['color'].map({'CACECB': 'серебристый', 'FAFBFB':'белый', 'EE1D19':'красный', '97948F':'серый', 
                                     '660099':'пурпурный', '040001':'чёрный','4A2197':'фиолетовый', 
                                     '200204':'коричневый','0000CC':'синий', '007F00':'зелёный', 'C49648':'бежевый',
                                     '22A0F8':'голубой','DEA522':'золотистый','FFD600': 'жёлтый', 'FF8649':'оранжевый',
                                     'FFC0CB':'розовый'})
  # Руль
    train['Руль'] = train['Руль'].map({'LEFT': 'Левый', 'RIGHT': 'Правый'})
  # ПТС
    train['ПТС'] = train['ПТС'].map({'ORIGINAL': 'Оригинал', 'DUPLICATE': 'Дубликат'}).fillna('Оригинал')
  # Владение
    train['Владение'] = train['Владение'].fillna('nodata')
    def months_to_sent(months):
        if months == 1:
            return f'{months} месяц'
        elif 2 <= months <= 4:
            return f'{months} месяца'
        return f'{months} месяцев'
    def years_to_sent(years):
        if 11 <= years <= 14 or 5 <= years%10 <= 9 or years%10 == 0:
            return f'{years} лет'
        elif years%10 == 1:
            return f'{years} год'
        elif 2 <= years%10 <= 4:
            return f'{years} годa'
    def tenure(row):
        row = re.findall('\d+',row)
        if row != []:
            years = 2020 - (int(row[0])+1)
            months = 2 +(12 - int(row[1]))
            if years < 0:
                return months_to_sent(int(row[1]))
            elif years == 0 and months < 12:
                return months_to_sent(months)
            elif years >= 0 and months == 12:
                return years_to_sent(years + 1)
            elif years >= 0 and months > 12:
                return years_to_sent(years + 1)+' и '+months_to_sent(months - 12)
            elif years > 0 and months < 12:
                return years_to_sent(years)+' и '+months_to_sent(months)
            return None
    train['Владение'] = train['Владение'].apply(tenure)
    return train

In [8]:
def test_(test):
    
  # удалим ненужные столбцы
    test = test.drop(['id','Состояние', 'Таможня', 'vehicleTransmission'], axis = 1)
  # приравняем к единому числовому варианту
    for feature in ['modelDate', 'numberOfDoors', 'productionDate', 'mileage']:
        test[feature] = test[feature].astype('int64')
  # vehicleConfiguration, выведем только трансмиссию
    test['vehicleConfiguration'] = test['vehicleConfiguration'].apply(lambda x: x.split()[1])
    test['engineDisplacement'] = test['engineDisplacement'].apply(lambda x: x.split()[0])
  # enginePower, оставим только цифру в enginePower
    test['enginePower'] = test['enginePower'].apply(lambda x: int(x.split()[0]))
  # Владельцы, оставим только цифру
    test['Владельцы'] = test['Владельцы'].apply(lambda x: int(x.split()[0]))
    return test


In [9]:
train = train_(asd_99)
train.dropna(axis = 0, thresh=18,inplace=True)#строки , где слишком много пропущенных значений.
test = test_(a)
pd.concat([param_data(train), param_data(test)], axis=1, sort=False)

,dtypes,nunique,isna,loc[0],dtypes,nunique,isna,loc[0]
bodyType,object,23,0,седан,object,23.0,0.0,внедорожник 5 дв.
brand,object,13,0,AUDI,object,13.0,0.0,MERCEDES
color,object,16,0,чёрный,object,16.0,0.0,синий
fuelType,object,5,0,бензин,object,5.0,0.0,бензин
modelDate,float64,70,0,1990,int64,51.0,0.0,2015
name,object,3490,0,2.3 MT (133 л.с.),object,2528.0,0.0,250 2.0 AT (211 л.с.) 4WD
numberOfDoors,float64,5,0,4,int64,4.0,0.0,5
productionDate,int64,73,0,1991,int64,49.0,0.0,2017
vehicleConfiguration,object,4,0,MECHANICAL,object,4.0,0.0,AUTOMATIC
engineDisplacement,object,431,0,2.3,object,53.0,0.0,2.0


In [10]:
train.Price=train.Price.apply(lambda x: np.log(x))

In [11]:
train['sample'] = 1
test['sample'] = 0
test['Price'] = 0
data = test.append(train, sort=False).reset_index(drop=True)
param_data(data)


,dtypes,nunique,isna,loc[0]
bodyType,object,23,0,внедорожник 5 дв.
brand,object,13,0,MERCEDES
color,object,16,0,синий
fuelType,object,5,0,бензин
modelDate,float64,70,0,2015
name,object,3608,0,250 2.0 AT (211 л.с.) 4WD
numberOfDoors,float64,5,0,5
productionDate,int64,73,0,2017
vehicleConfiguration,object,4,0,AUTOMATIC
engineDisplacement,object,437,0,2.0


In [12]:
data = test.append(train, sort=False).reset_index(drop=True)

param_data(data)

,dtypes,nunique,isna,loc[0]
bodyType,object,23,0,внедорожник 5 дв.
brand,object,13,0,MERCEDES
color,object,16,0,синий
fuelType,object,5,0,бензин
modelDate,float64,70,0,2015
name,object,3608,0,250 2.0 AT (211 л.с.) 4WD
numberOfDoors,float64,5,0,5
productionDate,int64,73,0,2017
vehicleConfiguration,object,4,0,AUTOMATIC
engineDisplacement,object,437,0,2.0


In [13]:
data['description'] = data['description'].fillna('[]')
data['description_len'] = data['description'].apply(lambda x: len(x.split()))
data['description_word'] = data['description'].apply(lambda x: [str(i).lower() for i in x.split()])

In [14]:
data['description_word'][300]

['обратившись',
 'в',
 'наши',
 'дилерские',
 'центры',
 'вы',
 'можете',
 'получить',
 'предложение',
 'по',
 'кредиту',
 'на',
 'автомобили',
 'нашей',
 'компании.',
 'возможен',
 'выгодный',
 'взаимозачет',
 'вашего',
 'автомобиля',
 'любой',
 'марки.',
 'на',
 'складе',
 'нашей',
 'компании',
 'более',
 '500',
 'автомобилей',
 'с',
 'пробегом.',
 'подробности',
 'уточняйте',
 'у',
 'менеджеров',
 'по',
 'продажам',
 'автомобилей',
 'с',
 'пробегом.',
 'все',
 'наши',
 'автомобили',
 'на',
 'площадке',
 'ао',
 'abtodom',
 'мкад',
 '51',
 'вы',
 'можете',
 'посмотреть,',
 'нажав',
 'на',
 'ссылку',
 '"продавец',
 'abtodom"',
 'в',
 'контактной',
 'информации.']

In [15]:
data['leather']= data['description_word'].apply(lambda x: 
                                                1 if ('темный' and 'салон') in x else 0)
data['carter']= data['description_word'].apply(lambda x: 
                                                1 if ('защита' and 'картера') in x else 0)
data['ABS']= data['description_word'].apply(lambda x: 
                                                1 if ('антиблокировочная' and 'система') in x else 0)
data['airbags']= data['description_word'].apply(lambda x: 
                                                1 if ('подушки' and 'безопасности') in x else 0)
data['immob']= data['description_word'].apply(lambda x: 
                                                1 if ('иммобилайзер') in x else 0)
data['central_locking']= data['description_word'].apply(lambda x: 
                                                1 if ('центральный' and 'замок') in x else 0)
data['on_board_computer']= data['description_word'].apply(lambda x: 
                                                1 if ('бортовой' and 'компьютер') in x else 0)
data['cruise_control']= data['description_word'].apply(lambda x: 
                                                1 if ('круиз-контроль') in x else 0)
data['climat_control']= data['description_word'].apply(lambda x: 
                                                1 if ('климат-контроль') in x else 0)
data['multi_rudder']= data['description_word'].apply(lambda x: 
                                                1 if ('мультифункциональный' and 'руль') in x else 0)
data['power_steering']= data['description_word'].apply(lambda x: 
                                                1 if ('гидроусилитель' or 'гидро' or 'усилитель' and 'руля') in x else 0)
data['light_and_rain_sensors']= data['description_word'].apply(lambda x: 
                                                1 if ('датчики' and 'света' and 'дождя') in x else 0)
data['сarbon_body_kits']= data['description_word'].apply(lambda x: 
                                                1 if ('карбоновые' and 'обвесы') in x else 0)
data['rear_diffuser_rkp']= data['description_word'].apply(lambda x: 
                                                1 if ('задний' and 'диффузор') in x else 0)
data['door_closers']= data['description_word'].apply(lambda x: 
                                                1 if ('доводчики' and 'дверей') in x else 0)
data['rear_view_camera']= data['description_word'].apply(lambda x: 
                                                1 if ('камера' or 'видеокамера' and 'заднего' and 'вида') in x else 0)
data['amg']= data['description_word'].apply(lambda x: 
                                                1 if ('amg') in x else 0)
data['bi_xenon_headlights']= data['description_word'].apply(lambda x: 
                                                1 if ('биксеноновые' and 'фары') in x else 0)
data['from_salon']= data['description_word'].apply(lambda x: 
                                                1 if ('рольф' or 'панавто' or 'дилер' or 'кредит' or 'ликвидация') in x else 0)
data['alloy_wheels']= data['description_word'].apply(lambda x: 
                                                1 if ('легкосплавные' or 'колесные' or 'диски') in x else 0)
data['parking_sensors']= data['description_word'].apply(lambda x: 
                                                1 if ('парктроник' or 'парктронник') in x else 0)
data['dents']= data['description_word'].apply(lambda x: 
                                                1 if ('вмятины' or 'вмятина' or 'царапина' or 'царапины' or 'трещина') in x else 0)
data['roof_with_panoramic_view']= data['description_word'].apply(lambda x: 
                                                1 if ('панорамная' and 'крыша') in x else 0)

In [16]:
vectorizer = CountVectorizer()
text_feat = vectorizer.fit_transform(data['description'])
data['mean'] = text_feat.mean(axis=1)
data['sum'] = text_feat.sum(axis=1)

In [17]:
data['name_1'] = data['name'].apply(lambda x: str(x).split()[0])

In [18]:
spis_eng = [str(i) for i in data['engineDisplacement'].unique()]


In [19]:
def in_eng_duspl(x):
    if x in spis_eng:
        return None
    else:
        return x

In [20]:
def engineDisplacement(row):
    row = str(row)
    engine = re.findall('\d\.\d', row)
    if engine == []:
        return None
    return float(engine[0])

In [21]:
data['engineDisplacement'] = data['name'].apply(engineDisplacement)

In [22]:
data['Владение'].value_counts()

1 год и 4 месяца      972
11 месяцев            903
1 год и 2 месяца      885
2 месяца              855
1 год и 1 месяц       838
                     ... 
26 лет и 4 месяца       1
25 лет и 6 месяцев      1
21 год                  1
24 года и 2 месяца      1
24 годa и 1 месяц       1
Name: Владение, Length: 334, dtype: int64

In [23]:
data['Владение_is_none'] = pd.isna(data['Владение']).astype('uint8')
data['Владельцы'] = data['Владельцы'].fillna(3.0)
data['engineDisplacement'] = data['engineDisplacement'].fillna(2.0) 

In [24]:
def num_of_months(row):
    if pd.notnull(row):
        list_ownership = row.split()
        if len(list_ownership) == 2:
            if list_ownership[1] in ['год', 'года', 'лет']:
                return int(list_ownership[0])*12
            return int(list_ownership[0])
        return int(list_ownership[0])*12 + int(list_ownership[3])

In [25]:
data['month_ownership'] = data['Владение'].apply(num_of_months)


In [26]:
data['month_ownership'].fillna(data['month_ownership'].median(),inplace=True)

In [27]:
data.drop(['description', 'Комплектация', 'name', 'description_word', 'Владение'], axis = 1, inplace = True)
print('Количество пропусков: ', data.isna().sum().sum())

Количество пропусков:  0


In [28]:
data = pd.get_dummies(data, columns = ['bodyType', 'brand', 'color', 'fuelType', 'vehicleConfiguration',
                                           'Привод', 'Руль', 'Владельцы', 'ПТС', 'engineDisplacement', 'name_1'], dummy_na=True)

In [29]:
data['modelDate'] = 2020 - data['modelDate']
data['productionDate'] = 2020 - data['productionDate']
data['prod-model'] = data['productionDate'] - data['modelDate']

In [30]:
train_data = data.query('sample==1').drop('sample', axis = 1)
test_data = data.query('sample==0').drop(['sample', 'Price'], axis = 1)

train_data = data.query('sample==1').drop('sample', axis = 1) # отделяем трейновые и тестовые данные
test_data = data.query('sample==0').drop(['sample', 'Price'], axis = 1).reset_index(drop=True)

In [31]:
X = train_data.drop(['Price'], axis=1).reset_index(drop=True)
y = train_data['Price'].values # target

In [32]:
models = [RandomForestRegressor(n_estimators =250,random_state = RANDOM_SEED, n_jobs = -1, verbose = 1),
         BaggingRegressor(ExtraTreeRegressor(random_state=RANDOM_SEED), random_state=RANDOM_SEED)]

def stacking_model_predict(models, X, y, test_data, sample_submission):
    for model_ in tqdm(models):
        model_.fit(X, y)
        pred_subm = model_.predict(test_data)
        sample_submission[str(model_)[:6]] = pred_subm
        sample_submission[str(model_)[:6]] = sample_submission[str(model_)[:6]].apply(lambda x: np.exp(x) )
        sample_submission[str(model_)[:6]] = sample_submission[str(model_)[:6]].apply(lambda x: round(x/1000)*1000)
    sample_submission['price'] = sample_submission.iloc[:,2:].mean(axis=1)
    sample_submission[['id', 'price']].to_csv(f'submission_v{VERSION}.csv', index=False)
    sample_submission.head(10)

stacking_model_predict(models, X, y, test_data, sample_submission)

  0%|          | 0/2 [00:00<?, ?it/s][Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  42 tasks      | elapsed:   55.4s
[Parallel(n_jobs=-1)]: Done 192 tasks      | elapsed:  4.1min
[Parallel(n_jobs=-1)]: Done 250 out of 250 | elapsed:  5.3min finished
[Parallel(n_jobs=4)]: Using backend ThreadingBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done  42 tasks      | elapsed:    0.3s
[Parallel(n_jobs=4)]: Done 192 tasks      | elapsed:    1.2s
[Parallel(n_jobs=4)]: Done 250 out of 250 | elapsed:    1.5s finished
100%|██████████| 2/2 [05:56<00:00, 178.19s/it]


In [33]:
sample_submission

,id,price,Random,Baggin
0,0,2229000.0,2313000,2145000
1,1,1485500.0,1506000,1465000
2,2,456000.0,467000,445000
3,3,3288000.0,3461000,3115000
4,4,1134000.0,1063000,1205000
...,...,...,...,...
35985,35985,544000.0,549000,539000
35986,35986,2782000.0,2799000,2765000
35987,35987,499500.0,503000,496000
35988,35988,804500.0,828000,781000
